# IBM & DataStax Demo - Banking AI Agent

## Part II - Langflow RAG

- Create a Langflow RAG flow and connect to Astra
- Use a IBM WatsonX.AI model
- Run the Flow through the Langflow API

# Langflow RAG

The Langflow part of our demo will be done on DataStax Langflow.

Once you log into Astra Dashboard, you will find the option to access Langflow on the top of the screen:

> INFO: DataStax Langflow, a managed service to provide Langflow "as-a-service" is currently in preview.
>
> But we can use it for demos and development.

<img src="img/access_lf2.png" alt="Langflow" width="600"/>

Once you access Langflow, you'll notice a sidebar on the left with folders available to help you organize your flows.
In the main area, you'll see all the flows contained within the selected folder.
To create a new flow, simply click the "+ New Flow" button.

<img src="img/lf_dash.png" alt="Langflow" width="800"/>

After clicking on the  "+ New Flow", you will see the available templates. Select "Vector Store RAG"

<img src="img/lf_templates.png" alt="Langflow" width="800"/>

This is the template flow create for a RAG operation. 

Note the components on the left side bar and how the components are connected.

You can zoom in and out to check the details.

<img src="img/lf_rag.png" alt="Langflow" width="800"/>

> DEMO TIP: If you are presenting Langflow to developers, select a component and present its code. Developers will feel more confortable by being able to edit the components and customizing it.
> 
> - Mouse over
>
> <img src="img/lf_comp_code.png" alt="Langflow" width="200"/>
> 
> - Checking component code
>
> <img src="img/lf_comp_code_2.png" alt="Langflow" width="600"/>
>
> Highlight to the developer it is possible the create new components and edit the existing ones to fit into the company needs.


The little flow below is responsible for loading data. As we already loaded the documents in the section before, you can delete this part. Just select all the components and hit _delete_.

> DEMO TIP: While demonstrating Langflow, especially with people who are not familiar with RAG, it is good to explain this part and how important it is.

<img src="img/lf_rag1.png" alt="Langflow" width="800"/>

Now, lets connect this flow to the loaded data.

- On the "Astra DB" component:
- - Select the database "ibm_astra_demo" (if it is not appearing, click on Refresh list inside the options)
- - Then, select the collection "banking_knowledge_layer" 
- - As this collection is using __Vectorize__, the "OpenAI Embedding"will be disconnected. You can now delete it.

You will have something like this:

<img src="img/lf_rag2.png" alt="Langflow" width="800"/>

- Click on the "Play" button on the top of the component to check the results clicking on the "DataFrame" output

<img src="img/lf_rag3.png" alt="Langflow" width="800"/>

Now, move to the "Prompt" component, Note the variables {context} and {question}.

- The context variable is being filled with the content of the Astra DB Vector Search.

- The question variable is being filled with the content of "Input chat", which receives the user input.

<img src="img/lf_rag5.png" alt="Langflow" width="200"/>

To edit the prompt or add variables, double-click on the "Template". It will be presented a screen to edit the prompt content and variables.


> DEMO TIP: It is possible to add more variables to make the prompt even more personalized.
>
> Just add {my_variable} to any part of the prompt and a new variable will be available to receive a value

<img src="img/lf_rag4.png" alt="Langflow" width="400"/>



### If you have an OpenAI Key

The template is using a "OpenAI" component to run the flow.

Note that the component receives the result of the prompt as "Input" and retuns a "Message" to the "Chat Output"

If you have an OpenAI API Key, you can enter it on "OpenAI API Key" field. You can also add a new system variable and share the same API Key with other flows.

<img src="img/lf_rag7.png" alt="Langflow" width="800"/>

### If you want to use and WatsonX.AI Model

You will need:

- Watsonx Project ID
- Watsonx API Key

Then, find the "IBM watsonx.ai" on the component bar and drag it to the screen.

<img src="img/lf_rag8.png" alt="Langflow" width="500"/>

- Delete the "OpenAI" component.
- Connect the prompt message from the promt command to the Input of the watson component.
- Connect the Message output from the watson component to the Chat Output text.
- Enter the project ID and the API Key on the watson component (You can save these values on system variables, if needed).

<img src="img/lf_rag10.png" alt="Langflow" width="800"/>




# Running the Flow

Click on the "Playground" on thw top right of the screen.

On the playground, you can type questions, like:

- Whats the refund limit for baggage lost?

> DEMO TIP: Depending on the model you are using, the answer can be slow, from 4 to 40 seconds.

<img src="img/lf_rag11.png" alt="Langflow" width="600"/>

Then close the playground and check the prompt output:

<img src="img/lf_rag12.png" alt="Langflow" width="600"/>

In fact, the context provided didn't mention a limit for refund. So, the model don't hallucintated, which is one of the main goals of implementing RAG.

Let's ask about baggage delayed:

- What about baggage delayed

<img src="img/lf_rag13.png" alt="Langflow" width="600"/>

Now it mention the limits for refunding.

> DEMO TIP: The answer may very between executions, so be aware to check the response 

## Integrating with an API

The flow can be executed through an API call.

Click on the Publish > API Access option on the top right of the screen.

<img src="img/lf_rag14.png" alt="Langflow" width="500"/>

<img src="img/lf_rag15.png" alt="Langflow" width="500"/>

Generate a Token and save it to the ".env" file 

```bash
LANGFLOW_API_TOKEN="AstraCS:..."
```

<img src="img/lf_rag16.png" alt="Langflow" width="500"/>


```bash
LANGFLOW_URL="AstraCS:..."
```

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)
LANGFLOW_TOKEN = os.getenv("LANGFLOW_API_TOKEN")

if LANGFLOW_TOKEN[:8] != "AstraCS:":
    raise ValueError("Environment variable LANGFLOW_API_TOKEN invalid format")

LANGFLOW_URL = os.getenv("LANGFLOW_URL")

if ".apps.astra.datastax.com" not in  LANGFLOW_TOKEN[:8] != "AstraCS:":
    raise ValueError("Environment variable LANGFLOW_API_TOKEN invalid format")

print(f'Langflow Token: {LANGFLOW_TOKEN[:10]}...{LANGFLOW_TOKEN[-5:]}')
print(f'Langflow URL: {LANGFLOW_URL[:10]}...{LANGFLOW_URL[-5:]}')
print("Good to go!")

Langflow Token: AstraCS:CR...4de30
Langflow URL: https://ap...a6d1b
Good to go!


In [27]:
# Function to run Langflow flow
import requests

def run_langflow_flow(question, langflow_url, token):
    """
    This function sends a request to the Langflow API to run a specific flow.
    It requires an API token and the flow ID to be specified in the URL.
    The function handles the request and prints the response.
    """
    # Request payload configuration
    payload = {
        "input_value": question,  # The input value to be processed by the flow
        "output_type": "chat",  # Specifies the expected output format
        "input_type": "chat"  # Specifies the input format
    }

    # Request headers
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token}"  # Authentication key from environment variable'}
    }

    try:
        # Send API request
        response = requests.request("POST", langflow_url, json=payload, headers=headers)
        response.raise_for_status()  # Raise exception for bad status codes
        json = response.json()  # Parse JSON response
        
        

        # Print response
        return json["outputs"][0]["outputs"][0]["results"]["message"]["data"]["text"]  # Access the output from the response

    except requests.exceptions.RequestException as e:
        print(f"Error making API request: {e}")
    except ValueError as e:
        print(f"Error parsing response: {e}")
    

In [29]:
# The Langflow API URL has a short timeout. If the execution fail, run it again.
from pprint import pprint
from textwrap import fill

# Replace the url with your Langflow API URL
url = f"https://api.langflow.astra.datastax.com/lf/ee339843-90eb-4fce-930c-5bab2ca033e1/api/v1/run/9ab891ce-09cd-45a6-966b-e38b23ba6d1b"

#Set your question here
QUERY = "Whats the limit for refund a laptop stolen?"
  
response = run_langflow_flow(QUERY, url, LANGFLOW_TOKEN)
# Wrap the response text to fit the screen width
if response:
    wrapped_response = fill(response, width=80)
    print(wrapped_response)
else:
    print("No response received.")

Based on the provided texts, Return Protection offers reimbursement for stolen
items up to $500 per item and an annual maximum of $1,000 per Account. However,
there are exclusions such as medical equipment, perishable or consumable items,
negotiable instruments, used or pre-owned items (unless refurbished with a
warranty), and specific conditions for vehicles or digital data. Since a laptop
isn't explicitly excluded in these categories and assuming it's a covered
purchase under Return Protection, the limit for refund in case of theft would be
$500.
